In [ ]:
import six, re
from itertools import product
import numpy as np

In [145]:
DATA = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
""".strip()

In [211]:
DATA = """
Tile 1249:
...#......
#..#..#.##
##........
#.#.......
..........
#...###...
#..#......
#...##....
..........
.....#...#

Tile 1693:
..#..####.
#.........
##..#....#
#.....#..#
......#.##
#........#
.....##...
.#..##.#.#
##........
###..#....

Tile 1481:
....#####.
#....#..##
##..#....#
##..#...#.
#..#.#...#
..........
...#..#...
.#...#...#
#...#.....
..#..##.##

Tile 3169:
####...#.#
#.#.....#.
.#......##
..#.#....#
...###..#.
#....###.#
.......#..
##.##.....
.#.#......
.#....#...

Tile 1229:
##..###..#
#.......#.
..#..##..#
##..#.....
#.#..#..#.
.#..#.#.##
....#....#
#..#..#.##
....#....#
..##..####

Tile 1489:
#......#..
....#.....
#.....#..#
#.......#.
#.#..#..#.
#.........
#........#
#..#.#....
#.........
....####.#

Tile 2477:
#####.##.#
.###...###
#....#..##
.#.#..#..#
###.##...#
#........#
#..#..#...
.......#.#
#......###
##.##.##.#

Tile 2897:
##.##..#.#
#.......##
#.#..#.#..
..#...#..#
...##.#..#
..#.......
#.#..##..#
..#....#.#
#....#.#.#
#......###

Tile 2083:
..#...##.#
###.#.##..
....##....
#...#..#..
...##....#
#..#......
.#.##.....
..##..####
....###..#
.#...#.#..

Tile 1069:
..#.#.#..#
..#....#.#
.........#
##..#.....
#.....#...
..##......
#..#......
.##..#####
#.#....#..
.....#..#.

Tile 1427:
...####.##
.###......
.#..#.#..#
..#.###.##
.#..#.....
..##.#....
.#......#.
#....#...#
.......#..
#.#..###..

Tile 1429:
.##.#.#.#.
#..##....#
..#......#
...#....##
...#.##..#
..#.#.....
#....#..##
#..#.....#
.##....#..
##.#.#...#

Tile 2357:
#.##..#.##
.........#
#..#.#..##
#....#.#..
#........#
#...##....
#....#....
....##...#
#.#..##...
.###.#.#..

Tile 3181:
...#..####
........#.
#...#...##
#.#.....##
#.........
##...#....
#.##.....#
#....#...#
..#.#...##
#..###.##.

Tile 2887:
.#..#.###.
#.........
.#....#...
#........#
..#.##...#
.......##.
....#.#..#
#...###..#
.#...#..##
###...#..#

Tile 2837:
##.##..###
.......#.#
....#.#..#
.......##.
.....#....
#.#.##..#.
#.#..#...#
....#.##..
..#......#
...######.

Tile 2539:
.###....#.
.......##.
##.##...#.
.###.....#
###..##...
.##....#.#
........#.
..........
.#........
#.....#...

Tile 2399:
##.#..##..
#...#.#..#
.##......#
###..##...
.#....#...
....#.#...
.....###.#
#.......##
#......#..
.##.##....

Tile 2383:
#..##.###.
.#.......#
#.#..#....
#..#.#...#
#.#.#.....
......#...
#....#..#.
..#.#.#..#
#..#.##..#
.#.##...##

Tile 2521:
.#..#.##.#
#........#
#.#...#.##
#........#
##.#.#..##
#..######.
.##..##..#
.#.....##.
.#.#......
..#..#####

Tile 1823:
#.#...##..
#..#.....#
.##......#
#...###.#.
......#...
.....##...
#....##.#.
#.........
#..#.....#
#.#.##.#..

Tile 1301:
####.#.###
#.##...##.
..#...####
..........
#....#..##
....#..#..
#..#.#.#..
...#...#..
###..###.#
.##..#.#.#

Tile 1289:
#.###....#
#.#.....#.
#.....##.#
#........#
.##.#...#.
.###..#...
..#.......
.##...#...
...#....##
.###....#.

Tile 3823:
#.##.##..#
.........#
#...##.#.#
.#.##.....
##.....#..
.#..#....#
#...#...##
#.........
#........#
.#.##.####

Tile 2411:
#.####.###
#....#.###
..##.....#
#.#...#..#
##...#.#.#
##.##...##
#.......#.
#...##...#
.........#
###.##.##.

Tile 1039:
.......#.#
..#.......
........#.
.##...####
##...#..##
.#.......#
..#.#.....
...#.....#
.........#
..#...#...

Tile 1609:
.##..#..##
##.###.###
.#.....#..
#........#
...#...#..
#.......##
...#..###.
#.##.##.##
#.......#.
...#.#....

Tile 2017:
##..#..#..
....###...
..##......
#...#.###.
#....#....
#.#.......
#..#.#....
###..##..#
###.#....#
#.#.#.#.##

Tile 3301:
.#####...#
...##.....
#..###...#
#....#...#
...#.#...#
#......#.#
#........#
...#......
.......#.#
..#####...

Tile 3733:
#####.##.#
#.##..#..#
#....#..##
#....#....
.....##...
#......#..
#....##.##
.........#
.#..#....#
.....##...

Tile 2309:
#..##.##..
...#.#.#.#
#..##.#..#
....##....
##....#..#
##..##.#.#
.###....#.
###...#.##
....#.....
.#.#.###.#

Tile 2879:
#.##.##...
...#.....#
...##.####
...#.#.#.#
#.###.##.#
.....#...#
#.....#.#.
#..#.#..##
.....###..
.##..#.###

Tile 3583:
.#.##...#.
..##...##.
#.#....##.
#.##...#.#
.....#..##
#####.....
..###..#..
..##..#.#.
.##....##.
##.##...#.

Tile 2153:
##...####.
..#.#...#.
##..##..#.
.#..#.#...
..........
..#...#...
#....#.###
...##....#
#.##.....#
#...#....#

Tile 3581:
#...##.###
..####..##
...##....#
.#..#.....
.##......#
#.#......#
...###.#.#
###.#.....
#.##.##...
#.#.....#.

Tile 2927:
##.#.#..##
..........
..........
.....#....
#..#..#..#
....#.#...
.#...#...#
.#...##..#
......#...
###.#.##.#

Tile 2861:
.#..#.###.
#..#....##
#.#.#..#..
........#.
..#.#.#...
...##....#
#..###.#.#
#..#..##..
..#.#....#
.#..####.#

Tile 2851:
#.#....#..
#........#
.#........
#..##.#...
..##....##
.#.......#
#.........
......#..#
..........
###.##..##

Tile 3319:
...##....#
#.......##
##.#..#...
.##....###
#...#...##
..#.......
.###..#..#
#...#.....
.........#
.#..##.#..

Tile 2143:
...#......
##.#...#.#
##.......#
#.##..#..#
..#...#..#
.#.#...#..
..#....#..
......#...
#.#.......
#..#..#.##

Tile 1093:
#.#...#.#.
##..#.....
.........#
##..#...##
##..#..#..
##..##....
##.##....#
#.#....#..
.#.#..#..#
..#.##.###

Tile 3391:
#....#..#.
#...#..#..
##....##..
..#.#..#.#
#####...##
..#..#...#
#.#......#
.#...#....
.....##.##
.#.#.#....

Tile 3917:
...##.#...
#..#.....#
#..##.###.
#.#..#..#.
.####..#.#
..##.#.###
#.#..##.#.
#....##..#
..###.#..#
..##.####.

Tile 1847:
#.##.#..#.
.#.##.....
#.#..#..#.
#....#.#..
.#.......#
..#..#.#.#
..##....##
#.#.##....
##.#..#...
.##.##.###

Tile 1667:
.##..#..#.
###....##.
#.####.#.#
...#.....#
.#.#..##.#
.#..#....#
#.##.#...#
#.#.#.....
...###...#
#.#..####.

Tile 1217:
...#####..
....#..#.#
..#...####
#........#
#........#
###..#...#
.##.##...#
.#.......#
.....##...
.##.#.###.

Tile 3467:
###.##....
.......#.#
.....#..##
##..#....#
....#..#..
....#..###
#......###
#.##...#..
..#.....##
...#.#..#.

Tile 1297:
..###..##.
#....#.#.#
.........#
###.#....#
#......#.#
##....##..
#.#.##..##
.##.....#.
.....#...#
.#.##.....

Tile 3877:
..###.#...
##..#....#
###.##..##
.#..######
....#.#..#
.#.......#
#...##.#..
..#..#....
#.##.##..#
#..##.#.#.

Tile 2389:
##...#####
##..#.#..#
#..#.....#
......#..#
.#........
#....##.#.
.##....#..
#.....#.#.
#........#
...##.#...

Tile 3361:
.....#.#..
.........#
...#....#.
##.##.....
#....##...
##...#...#
#...####.#
..##.....#
...#...#.#
###.####..

Tile 2207:
###.##..#.
#..###....
......#..#
#........#
##....##.#
........#.
...#..#...
#....#..##
...####...
##........

Tile 1997:
..#.#..###
......#...
...#...#..
..#......#
.........#
#....#...#
....#..#..
.....#....
#........#
#....#####

Tile 1063:
#.#...##..
.##..#....
#.#..##...
..#...####
......##..
#...#.#...
####......
.#.....#..
.#.......#
#...####..

Tile 3109:
...###.###
.......#..
......##.#
.#......##
#..##..#..
##..#..#.#
.....##.##
#....#.#.#
.....###..
####....#.

Tile 1097:
#####.##..
.#...#.##.
..#..#....
#..#....#.
#.#....#.#
.#...#.#.#
#....#..##
#....#...#
...#...#..
...#.#..#.

Tile 1117:
#..##.#.##
#.#.....#.
...#.#...#
#.#..##.#.
#.###..#..
#....#...#
....#.....
.#...#...#
..##..#..#
.#.#.....#

Tile 2551:
.#..#.####
#...#.....
##....##.#
#....#....
.##.....#.
.#...#....
#....#.#..
......#...
##.#.#....
.#....#..#

Tile 2677:
.#.####..#
#.#.....##
...#.#....
.#...#...#
.##...#..#
##....#...
...##.#..#
##...#..##
##...#...#
##...###.#

Tile 1367:
.#.##...##
#.#..#....
....##.#..
.........#
.#....##.#
....#.##..
....##...#
#.#..#...#
#.#.#..##.
....#.##.#

Tile 1913:
.##..#..##
.#...##.##
..#.....#.
......#...
...#...##.
...####...
..#..#....
....#.#...
#.........
.####.####

Tile 1709:
.#....##..
..#...#.##
.#......#.
...#....##
.#.##...#.
.#.#.#.###
.........#
#.......#.
##....#...
..#.##.###

Tile 1459:
.##..#.#..
...##.....
.#.#...###
...#.....#
##..#..#.#
.........#
.........#
....#...#.
#..#......
...#.#...#

Tile 2137:
##..#....#
#.#####.#.
......#..#
#.#......#
#...#..#.#
#..#..#.##
###..####.
.##..#...#
#.#.......
.##.##....

Tile 2659:
#.##..#...
#...##....
#...#...#.
#........#
#....#....
.......##.
..#.....#.
#.#..##..#
#......#..
..###.#..#

Tile 2657:
####...##.
........#.
..####..#.
..#.......
....#.#..#
#.#.#.#...
...#....#.
###...####
........##
##.#.#.#..

Tile 2099:
#.###.##.#
#..#...###
..#......#
.#...#.#..
...##.....
###.##...#
..##..#...
...#......
..#.#.#.#.
..#.##....

Tile 3209:
##.###....
...#....#.
#..##..##.
##.##.#...
.#..#....#
#..#.#...#
.#...#...#
#..#......
...#......
..#.#.###.

Tile 1879:
#.##.#.##.
.#.##.#...
#..#.#....
#.....###.
#..##.....
....##....
#..#..#...
#.........
..#.##....
...####.##

Tile 1621:
###....#.#
.........#
.#....##..
.....#.#..
.#...#.#..
##...####.
.....#...#
#.........
....#.#..#
.####...##

Tile 3931:
.####.###.
#.##.....#
#..#.#...#
..#......#
###..#.##.
....##....
#..#......
..........
.#.....#.#
#.##.#..##

Tile 2777:
##....####
.#.#......
##......#.
#.......##
.#..#.#..#
##.#...#.#
##.#.....#
...##....#
#..#.#....
#.#..#....

Tile 2909:
#......##.
#........#
..#.#.###.
.......#.#
........##
....#.....
......#...
#........#
#.......#.
......#.#.

Tile 1777:
.##...####
.##.....#.
#..#.#..#.
.#......##
.#....#..#
#.###.#...
.#.#......
###..#...#
.#.......#
..#...#.##

Tile 3251:
...#.###..
#....##..#
#....#..#.
#....#..##
.....#...#
#.......##
.......#.#
#....#..#.
#...##...#
.#.#.....#

Tile 1601:
#....#....
.##.##...#
.........#
#.........
.#.....###
#..##.#.#.
#......#.#
.#..#.#..#
.#...#....
.#.#.#..##

Tile 1283:
#####.#..#
..#....##.
#..#......
.#....##..
.#.#.....#
##......##
#.....##..
##..#....#
#........#
.#.###..##

Tile 3079:
..#.##....
.......#.#
..#...##.#
..###...#.
#..#.....#
..##.....#
..#..#.#..
.##.#.#..#
#..#......
.#.##..#..

Tile 3793:
##..#####.
##...#....
##....#...
..........
#.........
.#....#..#
#....#..#.
#....#....
###..##...
#.###.##..

Tile 3037:
.##.#..##.
#..#.###..
.#....#...
...####..#
..#.###..#
#.#..#....
.##......#
#.....#..#
#...#.#...
##.######.

Tile 1669:
##.#......
.#......#.
##.#...#.#
#.#.#.#..#
##........
##....#...
#.#.....#.
..........
.........#
####...###

Tile 1087:
.###.#.#.#
###.......
.....#...#
##.....#..
.#.#......
...#......
#...#..#.#
#........#
#..#....##
.#...#####

Tile 2617:
.##..#.#..
.....#...#
..#.#.....
....#...##
####...##.
..##......
...#...#..
..#.#.##.#
#.#.......
###.##....

Tile 3943:
.#.####..#
.........#
#.........
.#........
###..#....
...#....#.
#...#.#...
.#......#.
...##.#...
..##....#.

Tile 2273:
##.#.#...#
...##.#..#
..##......
..#.....#.
#......#..
...#....#.
.##....#.#
...#.#.#..
#..###.#.#
###.#..##.

Tile 3803:
##....#..#
##...#.#.#
..##....#.
#...#..###
....#.....
....#.####
#.....##..
##....###.
#....#..##
#####.##..

Tile 1697:
....####.#
#....#.#.#
#.....#.##
#......#..
#.#...#...
#....#.#..
.##....#..
##...#.#.#
#.#..#..#.
.......#..

Tile 3343:
######.#..
#....##...
#.#.##...#
#...####..
#......#..
##.#..##.#
#.....##.#
..##.##..#
..#......#
###..##...

Tile 1549:
#.##.###.#
.....###..
...#.###..
##...##..#
#.#.......
#.........
....#....#
..#.....##
.....##...
#.#.##.##.

Tile 1619:
..###.###.
##.###...#
#...##...#
....##...#
..##..#.#.
#...#.#..#
#.......##
#..#.....#
....##...#
.#...#.#..

Tile 2971:
..##.###..
#.....#..#
#..#.....#
#..#...#.#
...#......
#......###
#....####.
.#....#...
...#.....#
#...###..#

Tile 3617:
####...#..
.......#.#
.#.#......
#..##..###
.##.......
.#....#..#
.....#.###
....##.#..
#....#...#
.#.##.####

Tile 1543:
......#.##
#......#.#
##.....###
#...#...##
#.#....#..
.#........
#....#....
......##..
##...#..#.
#...#.##.#

Tile 2111:
####..#...
#..#...#.#
#.#.#.##..
##........
#...#..#..
#.##..#...
#.........
##.#..#...
.##.#.###.
#.##.####.

Tile 1499:
#.####.#..
..##.#...#
#..#......
....#.....
.#..##...#
#.#...##.#
...#.##.##
###..#.###
.....#....
...##.##.#

Tile 3461:
.#####.#.#
#.........
..#..#....
..#......#
###...####
...#.##..#
.#........
#..#....##
#...#...##
####.....#

Tile 1483:
.....#.##.
##........
#.#.....#.
##......#.
#.....#...
..#.......
##....#..#
##........
..#...####
......###.

Tile 2467:
###.#.#...
#.#.....#.
#...#....#
#....#...#
...#..#..#
##...#....
#.#.###...
#..##..#.#
#.........
..######.#

Tile 3631:
###.#...##
.........#
#.##..#...
...##..#..
##..#.#..#
#..#.#....
......#.#.
..........
#......##.
..##.#....

Tile 3767:
#...#.#..#
..........
#......#.#
#...#.#..#
##.......#
..#......#
........#.
##...##.##
#........#
....#....#

Tile 2381:
.#.#.##...
#.##.#....
...##.#.##
#.....#.#.
#......#..
...#....#.
......####
#.#.#.....
##...##..#
...#...#.#

Tile 2687:
###...####
.....#.###
..#.#.#.#.
##..#...##
#.###...#.
..#.##..##
##.....#..
##..#..#.#
#...#..#.#
#.#..###.#

Tile 3719:
..####.#..
.#......##
#......###
##....###.
.#...##.##
.#.....#.#
..##..#.#.
#......###
...#...#..
......###.

Tile 3259:
######.#.#
#..#......
.........#
#.......#.
..##...#..
##...##.##
#......#.#
##.......#
#.......#.
#..##.#.##

Tile 3643:
###..#..##
#.#.#.#...
..##....#.
##....##..
..##......
#..#..#.##
......##..
#.......#.
#...##..##
#.#..#.###

Tile 2767:
####....##
..........
.........#
..#....#.#
#......###
#.##..#..#
#.#..#.#..
...#.#...#
#..####..#
##..#####.

Tile 2333:
###..#.##.
..##.....#
..#.#..##.
#.....#..#
....#.#...
##...#....
#.#......#
###.......
#...#....#
..#..#..##

Tile 2857:
..#..##..#
#.....#...
##....#..#
#...##....
....#.#..#
#.#.##....
#...##..#.
.#...##..#
#...#.#...
.#.###.#.#

Tile 1193:
#.#..#.#.#
##..####..
###..##.#.
#.......##
###....##.
...#...#..
#...#....#
##.......#
#####.##.#
..#####.##

Tile 2351:
.##..#####
#.........
.#..#.....
#.##.#...#
..#.#..#..
#.###....#
###..#....
##.......#
#...#..#..
.....#...#

Tile 1129:
..##..####
.....##..#
##........
.........#
#...#.....
....#...##
..#....##.
..#.....#.
..#.......
.##.###...

Tile 3433:
.##..#....
...###.###
.......##.
#..#.#.#.#
........#.
...###...#
...#......
.##...###.
.#.#..#..#
##..##..#.

Tile 1091:
###.####..
.#..#.....
##.......#
#.#...#...
...##.....
#..###...#
..##...#.#
#...#..###
.....#...#
###.#....#

Tile 3407:
.####.#...
.#..#.#..#
..#..#....
......#..#
.........#
#...##...#
...#.###.#
#....#...#
#.....###.
.#.#.#.#..

Tile 2591:
.##..##...
..#...##.#
...#..####
.#....#.##
#...##.#..
#...#....#
#..#.....#
#..#.....#
..........
.#.###.#.#

Tile 3613:
###.######
.#...##..#
#.##.....#
#.........
#...#.....
###.....##
.#.##.#..#
##..#...##
####......
..#...#...

Tile 3967:
#.#..#.#..
#..#.....#
...##...#.
.#.##...#.
#####..#.#
..##.#..#.
..........
#.#.#..#.#
#.#.....##
#..#####.#

Tile 1999:
#.#.#..##.
####.##.##
..##...#.#
....#....#
.##.#.#...
..........
.......##.
#.#......#
##...#.#.#
.#.#.#.#..

Tile 2689:
.#..#####.
.#........
..#....#.#
#......#.#
.#.#..#.##
###...#..#
...#...#.#
##......##
......##.#
.##.#..##.

Tile 3533:
.####.#...
#........#
#......#..
.........#
#..#.##..#
###....#.#
...##..#..
..#.....#.
#...#..###
..###.##.#

Tile 2267:
#.##...#.#
#....#....
.#.#.##...
.#....#.##
..#.#.....
.##.......
##.....#..
####....#.
......#..#
#..##..#.#

Tile 2297:
###..##...
#..##..##.
..#....#.#
#.#.##...#
..#.##..#.
...##...#.
#...#.....
.....#.#.#
#..#....#.
.###..##..

Tile 2711:
##..##...#
...##....#
...#..#.#.
#........#
...#.....#
........##
##....##..
#.##.....#
#.###.....
###...#..#

Tile 1931:
.###...###
......#.##
....##..##
#...#...#.
..#..##.#.
#.....#..#
#...#....#
#........#
...#..#.#.
#.##..###.

Tile 1787:
#...##..##
..#.......
#..#.#.#..
...#..#..#
.##.#..###
#.#.#.....
..........
....##...#
..........
.#.#...#..

Tile 2549:
##.#...###
.....#...#
#......###
##.###....
#..#....#.
##.......#
.#........
..###...##
##..##...#
#.##...#.#

Tile 2789:
#.#.#.#.##
#...#.##..
.##...#.##
#.##.#....
...#......
###.......
#.......##
#.#...#..#
..#.#....#
.##.#..#..

Tile 2707:
#..#.#...#
#.#.#.#...
...#..#..#
##.#.....#
#....#.##.
.....#...#
#.....#...
....#.#...
#..###...#
##..##.###

Tile 3313:
###.#..#..
.#....#..#
...#.....#
....#..###
....#..#..
#..#.#....
##.#..##..
..#...#.##
##..##....
.##.###.#.

Tile 1607:
..#...###.
#.....#...
#.......##
#.....#...
###.#.#..#
#.#.....#.
#..#...###
.........#
..#.......
##.#.#####

Tile 3889:
###.#...##
##.......#
....#..#..
#.#......#
.##......#
#...##....
#....##..#
.......#..
#...#...#.
##.#####.#

Tile 3821:
..###.##..
........##
#..##..#.#
.##.#..#.#
#....##.##
#.....##.#
#.....#..#
##........
#.#..#....
##.#.###..

Tile 3347:
.##..#...#
#.#...#...
#..#..###.
.#......#.
#..###.##.
......#...
##..##.#..
.#.###.#..
........#.
#.###..##.

Tile 1907:
.#.##...#.
#......#..
...#......
##.......#
.#.....#..
.#.....###
........##
.##.#..#.#
##.....#..
#.########

Tile 2903:
##.....#.#
....#.##..
...#.....#
.#..#####.
.....##.#.
.#......#.
..........
#..#.....#
....#...##
.#..##..##

Tile 1399:
#....###.#
..#..#####
.#....##.#
...#...#..
#..##.#..#
.....#....
..#......#
........##
#..###....
#..###.###

Tile 1993:
###.##..#.
##........
##..#..##.
.........#
.........#
#..##.#.#.
.##.......
#..#.....#
#.##.#..##
##.#.#..#.

Tile 3671:
...##.#..#
##....##.#
#..#....##
#..#...#.#
.#.#.#...#
........##
..........
##.......#
#.##....##
.#..#..##.

Tile 3167:
###..##..#
.....##...
.......#..
#...#.#..#
.#.##.#..#
###....##.
##........
#.........
..#..#....
.....##.#.

Tile 1109:
..#...#..#
#..#.#.#..
.#........
.......#.#
#..#..#...
.........#
##.#......
........##
#.#.##..#.
..#..#.#..

Tile 3089:
.###.#####
#..#..#.#.
#......#.#
...#.....#
.........#
.#.#...#..
.#..#.#.##
.......###
.#.#..#.##
..#..###..

Tile 1051:
.....##..#
#...##.#..
..........
..........
#.#.##..#.
...##...##
.##.#####.
.#...##..#
..#.#...#.
#.......##

Tile 2113:
#...###.##
.#.....##.
...#....##
...##..#..
#....#...#
..##.#....
##....#.#.
..#...###.
.....#.#.#
.##......#

Tile 2131:
#....#####
#.#.......
.###.#..##
###.##.#..
#...#.....
.#.#.....#
###....#..
..#.....##
.#...##.##
#..###....
""".strip()

In [212]:
from copy import deepcopy

In [224]:
class Solver:
    
    def __init__(self, data):
        
        self.n = int(np.sqrt(data.count('Tile')))
        self.height, self.width = 10, 10
        self.layout = np.zeros((self.n * self.height, self.n * self.width))
        self.references = [[None for _ in range(self.width)] for _ in range(self.height)]
        
        i, self.tiles = 0, {}
        while i < len(data.splitlines()):
            line = data.splitlines()[i]
            if line.startswith('Tile'):
                j = int(line.split('Tile ')[1].split(':')[0])
                grid = data.splitlines()[i+1:i+11]
                grid = np.array([[c == '#' for c in row] for row in grid])
                self.tiles[j] = grid
            i += 12        
    
    def neighbours(self):
        
        """
        For all given tiles, finds all the possible neighbouring
        tiles, considering all possible rotation and flips.
        
        :return: dict(
            tileIdentifier:int,
            neighboursPerDirection:list(
                topNeighbours:set(
                    neighbourIdentifier:int,
                    rotations:int,
                    horizontalFlips:int,
                    verticalFlip:int
                ),
            )
        )
        """
        
        matches = {}
        
        for i, a in six.iteritems(self.tiles):
            matches[i] = [set(), set(), set(), set()]
            for j, b in six.iteritems(self.tiles):
                if i == j: continue
                for r in [0, 1, 2, 3]:
                    b = np.rot90(b) if r else b
                    for fx, fy in product((0, 1), (1, 0)):
                        c = deepcopy(b)
                        c = np.flip(c, 0) if fx else c
                        c = np.flip(c, 1) if fy else c
                        if np.all(a[0,:] == c[-1,:]): matches[i][0].add((j, r, fx, fy))
                        elif np.all(a[:,-1] == c[:,0]): matches[i][1].add((j, r, fx, fy))
                        elif np.all(a[-1,:] == c[0,:]): matches[i][2].add((j, r, fx, fy))
                        elif np.all(a[:,0] == c[:,-1]): matches[i][3].add((j, r, fx, fy))
            
                    
        return matches

In [225]:
s = Solver(DATA)
neighbours = s.neighbours()

In [226]:
# Find all tiles which have at most two neighbours;
# while technically the true corners might have potential
# neighbours on the two outer edges, this has not 
# occurred in the examples and real input;
res = 1
for tile, neighbours in six.iteritems(neighbours):
    c = sum(bool(d) for d in neighbours)
    if c == 2:
        print(tile)
        res *= tile
res

2551
1697
1129
3313


16192267830719